# Mount and Imports

In [1]:
#drive.flush_and_unmount()

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
pip install wandb -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.8 MB/s eta 0:00:00


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.tensorboard as tb
from torch.nn import functional as F


import os
import numpy as np
import math
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm
import wandb

In [5]:
# Set the device to use for training
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [6]:
#!unzip '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_All/t.zip' -d '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_All/'

In [7]:
cd '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Prov/video_anomaly_diffusion-main'

/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Prov/video_anomaly_diffusion-main


# Diffusion

## Dataset and Utils

In [8]:
!pip install jsonmerge
!pip install torchdiffeq
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 3.2 MB/s eta 0:00:00


In [9]:
'''   Dataset   '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tqdm
import torch

'''   custom dataset with trajectories    '''
class CustomDataset(Dataset):
    def __init__(self, dataset_path):

        self.data = []
        self.trajectories = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(dataset_path)



    def __len__(self):
        return len(self.trajectories)


    def __getitem__(self, idx):
        class_id, trajectory = self.data[idx]

        trajectory = torch.Tensor(trajectory)

        # return both trajectory and class id
        return trajectory, class_id

    def _init_dataset(self, dataset_path):

        #list_behaviors = [item for item in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, item))]
        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        self.labels_codec.fit(list_behaviors)

        for root, dirs, filenames in os.walk(dataset_path, topdown=False):
            for filename in filenames:
                behavior = filename[:filename.find("_")]
                behavior_int = self.label_to_int(behavior)
                self.labels.append(behavior_int)

                filename_path = root + "/" + filename
                df = pd.read_csv(filename_path)

                trajectory = []
                for i in range(len(df)):
                    traj_point = df.loc[i].tolist()
                    trajectory.append(traj_point)

                self.trajectories.append(trajectory)

                data = [behavior_int, trajectory]
                self.data.append(data)

    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

In [10]:
'''   utils   '''

class Metric(object):
    def __init__(self, name):
        self.name = name
        self.sum = torch.tensor(0.)
        self.n = torch.tensor(0.)

    def update(self, val):
        self.sum += val.detach().cpu()
        self.n += 1

    @property
    def avg(self):
        return self.sum / self.n

def random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def log(s, nl=True):
    print(s, end='\n' if nl else '')

# loc = mean, scale = std
def find_sigma(loc, scale):
  sigma_min = np.exp(loc - 5 * scale)
  sigma_max = np.exp(loc + 5 * scale)

  return sigma_min, sigma_max

import random
def seed_everything(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

In [11]:
from contextlib import contextmanager
@contextmanager
def train_mode(model, mode=True):
    """A context manager that places a model into training mode and restores
    the previous mode on exit."""

    modes = [module.training for module in model.modules()]

    len_modes = len(modes)


    try:
        yield model.train(mode)
    finally:

        for i, module in enumerate(model.modules()):
            module.training = True
            #if i < len_modes:
              #module.training = modes[i]

In [12]:
def eval_mode(model):
    """A context manager that places a model into evaluation mode and restores
    the previous mode on exit."""
    return train_mode(model, False)

In [13]:
# imports
import k_diffusion as K
from collections import Counter
import accelerate
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy

## Train

In [14]:
import k_diffusion as K
from collections import Counter
import accelerate
import torch.nn as nn
from torch import optim
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy

def training_diffusion(config_tune, n_epochs = 2, dataset=None, batch_size = 32, seed = 3, grad_accum_steps=1, wandb_update = False):



    print("config")

    # it loads configurations from k_diffusion/config.py
    config_file_name = 'configs/config_ano.json'
    config = K.config.load_config(open(config_file_name))

    model_config = config['model']
    dataset_config = config['dataset']
    opt_config = config['optimizer']
    sched_config = config['lr_sched']
    ema_sched_config = config['ema_sched'] # ema = esponentially moving average

    model_config['sigma_max'] = config_tune["sigma_max"]
    model_config['sigma_min'] = config_tune["sigma_min"]
    model_config['sigma_data'] = config_tune["sigma_data"]
    model_config['sigma_sample_density']['mean'] = config_tune["ssd_mean"]
    model_config['sigma_sample_density']['std'] = config_tune["ssd_std"]
    opt_config['lr'] = config_tune["lr"]
    opt_config['weight_decay'] = config_tune["weight_decay"]

    print("dataset")



    feat = []
    labels = []
    for i in range(len(dataset)):
      array = dataset[i][0].numpy()
      feat.append(array)
      labels.append(dataset[i][1])


    print("features")

    print(len(feat))
    print(feat[0])
    print(feat[0][0])

    # feature size
    feat_size = 13

    print(feat_size)

    if model_config['input_size'] != feat_size:
      print('input size replace.')
      model_config['input_size'] = feat_size


    print("accelerators")
    # accelerators
    ddp_kwargs = accelerate.DistributedDataParallelKwargs(find_unused_parameters=model_config['skip_stages'] > 0)
    accelerator = accelerate.Accelerator(kwargs_handlers=[ddp_kwargs],
                                         gradient_accumulation_steps=grad_accum_steps)

    device = accelerator.device
    print(f'Process {accelerator.process_index} using device: {device}', flush=True)

    # seeds for accelerator
    if seed is not None:
        seeds = torch.randint(-2 ** 63, 2 ** 63 - 1, [accelerator.num_processes],
                              generator=torch.Generator().manual_seed(seed))
        torch.manual_seed(seeds[accelerator.process_index])

    print("model")
    feat_size = 13

    gvad_model = K.models.denoiser_model.GVADModel(
        feat_size,
    )

    # tensorboard
    if accelerator.is_main_process:
        from torch.utils.tensorboard import SummaryWriter
        print('Parameters:', K.utils.n_params(gvad_model))
        com_str = ''
        com_str += f'_lr:{config_tune["lr"]}'
        com_str += f'_s_max:{config_tune["sigma_max"]}'
        com_str += f'_s_min:{config_tune["sigma_min"]}'
        com_str += f'_s_data:{config_tune["sigma_data"]}'
        com_str += f'_ssd_mean:{config_tune["ssd_mean"]}'
        com_str += f'_ssd_std:{config_tune["ssd_std"]}'
        com_str += f'_weight_decay:{config_tune["weight_decay"]}'
        writer = SummaryWriter(comment=com_str)

    # optimizer
    if opt_config['type'] == 'adamw':
        opt = optim.AdamW(gvad_model.parameters(),
                          lr=opt_config['lr'],
                          betas=tuple(opt_config['betas']),
                          eps=opt_config['eps'],
                          weight_decay=opt_config['weight_decay'])
    elif opt_config['type'] == 'sgd':
        opt = optim.SGD(gvad_model.parameters(),
                        lr=opt_config['lr'],
                        momentum=opt_config.get('momentum', 0.),
                        nesterov=opt_config.get('nesterov', False),
                        weight_decay=opt_config.get('weight_decay', 0.))
    else:
        raise ValueError('Invalid optimizer type')

    if sched_config['type'] == 'inverse':
      sched = K.utils.InverseLR(opt,
                              inv_gamma=sched_config['inv_gamma'],
                              power=sched_config['power'],
                              warmup=sched_config['warmup'])
    elif sched_config['type'] == 'exponential':
        sched = K.utils.ExponentialLR(opt,
                                      num_steps=sched_config['num_steps'],
                                      decay=sched_config['decay'],
                                      warmup=sched_config['warmup'])
    else:
      raise ValueError('Invalid schedule type')


    # assert = checks if something is true or not
    assert ema_sched_config['type'] == 'inverse'
    ema_sched = K.utils.EMAWarmup(power=ema_sched_config['power'],
                                  max_value=ema_sched_config['max_value'])

    if accelerator.is_main_process:
        try:
            print('Number of items in dataset:', len(dataset))
        except TypeError:
            pass


    # dataloader
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    model, opt, train_dl = accelerator.prepare(gvad_model, opt, loader)
    inner_model = model.ae

    # sample density
    sigma_min = model_config['sigma_min']
    sigma_max = model_config['sigma_max']
    sample_density = K.config.make_sample_density(model_config)

    model_denoiser = K.config.make_denoiser_wrapper(config)(inner_model)
    model_ema = deepcopy(model_denoiser)


    # wandb
    if wandb_update:
      # save model inputs and hyperparameters
      config = {
        "lr": opt_config['lr'],
        "batch_size": batch_size,
        "epochs": n_epochs,
        "config": config_tune,
      }

      # start a new wandb run
      run = wandb.init(project='project_name', entity='entity_name', config=config)

      # log gradients and model parameters
      wandb.watch(model)



    step = 0


    print("training")
    # training

    for epoch in range(n_epochs):

        #trainMSE = Metric('trainMSE')
        for batch in tqdm(loader, disable=not accelerator.is_main_process):
            with accelerator.accumulate(model):

                reals = batch[0]

                # resize correctly the trajectory: batch size x 13 x 3
                reals = torch.transpose(reals, 2,1)

                # torch randn returns a tensor with the same size as input that is filled with random numbers from a normal distribution with mean 0 and variance 1
                # noise
                # noise sampling
                noise = torch.randn_like(reals)

                sigma = sample_density([reals.shape[0]], device=device)


                reals = reals.cuda()
                noise = noise.cuda()
                sigma = sigma.cuda()

                g_losses = model_denoiser.loss(reals, noise, sigma) # losses with the batch

                gen_dist = accelerator.gather(g_losses)

                loss = gen_dist.mean()

                accelerator.backward(loss)

                # update
                opt.step()
                sched.step()
                opt.zero_grad()

                #print("ema")

                if accelerator.sync_gradients:
                    ema_decay = ema_sched.get_value()
                    K.utils.ema_update(model_denoiser, model_ema, ema_decay)
                    #K.utils.ema_update(model, model_ema, ema_decay)
                    ema_sched.step()


            if accelerator.is_main_process:
                writer.add_scalar('Epoch/train', epoch, step)
                writer.add_scalar('Loss/train', loss.item(), step)
                writer.add_scalar('ema_decay/train', ema_decay, step)

                if step % 25 == 0:
                      tqdm.write(f'Epoch: {epoch}, step: {step}, loss: {loss.item():g}')


            step += 1

        print("epoch, step and loss", epoch, step, loss.item())


        @torch.no_grad()
        @eval_mode(model_ema)
        def evaluate(model, sigma_min, sigma_max, loader, n_start=0):

            tqdm.write('Evaluating...')

            sigmas = K.sampling.get_sigmas_karras(10, sigma_min, sigma_max, rho=7., device=device)
            sigmas = sigmas[n_start:]

            sigmas = sigmas.cuda()

            sample_noise = torch.randn([1, feat_size], device=device) * sigma_max
            sample_noise = sample_noise.cuda()

            def sample_fn(x_real, sigmas, model):
                #x_real = x_real.to(device)
                x_real = x_real.cuda()
                x_real = torch.transpose(x_real, 2,1)

                x = sample_noise + x_real

                x = x.cuda()
                sigmas = sigmas.cuda()
                model = model.cuda()

                x_0 = K.sampling.sample_lms(model_ema, x, sigmas, disable=True)
                return x_0

            def compute_eval_outs_aot(sample_fn, dl,sigmas,model):
                outputs = []
                test_loss = []

                for batch in tqdm(dl):
                    feat = batch[0]


                    g_dists = sample_fn(feat,sigmas,model_ema)

                    g_dists = accelerator.gather(g_dists)

                    feat = torch.transpose(feat, 2,1)
                    feat = feat.cuda()
                    g_dists = g_dists.cuda()

                    loss =  model.loss(feat, g_dists).mean().item()

                    test_loss.append(loss)

                    outputs.append(g_dists)


                g_dists = torch.cat(outputs)


                test_loss =  sum(test_loss)/len(test_loss)

                return test_loss

            test_loss = compute_eval_outs_aot(sample_fn, loader,sigmas,model)

            return test_loss


        test_loss = evaluate(model=model, sigma_min=model_config['sigma_min'], sigma_max=model_config['sigma_max'], loader=loader, n_start=0)

        print("Test Loss:", test_loss)


        # update wandb
        if wandb_update:
          wandb.log({'Epochs': epoch + 1,
                    'Step': step,
                    'Train Loss': loss.item(),
                    'Test Loss': test_loss})

        '''
        if test_loss < 0.0025:
          path = "/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/"
          name = "AATest" + str(100. * test_loss)
          torch.save({
              'epoch': epoch+1,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': opt.state_dict(),
              'model_ema': model_ema.state_dict(),
              'sched': sched.state_dict(),
              'ema_sched': ema_sched.state_dict(),
          }, f'{path}/{name}.pth')


        if loss.item() < 0.1:
          path = "/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/"
          name = "Train" + str(100. * loss.item())
          torch.save({
              'epoch': epoch+1,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': opt.state_dict(),
              'model_ema': model_ema.state_dict(),
              'sched': sched.state_dict(),
              'ema_sched': ema_sched.state_dict(),
          }, f'{path}/{name}.pth')

          '''

    # end wandb run
    if wandb_update:
      run.finish()

In [15]:
''' start training  '''

from tqdm import tqdm
from torch import multiprocessing as mp
import warnings
import k_diffusion.models.denoiser_model
def start(dataset):
  warnings.filterwarnings("ignore")
  print("initialization")


  epochs = 100
  batch_size = 512
  seed = 3

  # initialize random seeds
  seed_everything(seed)

  # select start method for multiprocess
  #mp.set_start_method('spawn')

  # A bool that controls whether TensorFloat-32 tensor cores may be used in matrix multiplications on Ampere or newer GPUs
  torch.backends.cuda.matmul.allow_tf32 = True

  std = [0.25]
  mean = [-3]
  for ssd_std in std:
    for ssd_mean in mean:
        sigma_min, sigma_max = find_sigma(ssd_mean, ssd_std)

        config = {
            "sigma_min": sigma_min,
            "sigma_max": sigma_max,
            "sigma_data": 0.2574,
            "ssd_mean": ssd_mean,
            "ssd_std": ssd_std,
            "lr": 0.002,
            "weight_decay": 0.0005,
        }


        training_diffusion(config, n_epochs = epochs, dataset = dataset, batch_size = batch_size, seed = seed, wandb_update = True)

In [ ]:
!wandb login

In [ ]:
dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/SLB'
# load dataset
dataset = CustomDataset(dataset_path)
print(len(dataset))

In [ ]:
start(dataset)

## Eval



In [ ]:
''' Setting same noise. If not evaluation is different every time as the noise is randomly extracted each time '''
seed_noise_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/Noise/noise.pth'
if os.path.exists(seed_noise_path):
      seed_noise = torch.load(seed_noise_path, map_location=device)
else:
      seed_noise = torch.randn([1, 13], device=device)
      torch.save(seed_noise, seed_noise_path)

In [ ]:
'''   imports   '''
import pandas as pd
from tqdm import tqdm
import k_diffusion.models.auto
import k_diffusion as K
import k_diffusion.config as con


config_file_name = 'configs/config_ano.json'
config = K.config.load_config(open(config_file_name))

model_config = config['model']
dataset_config = config['dataset']
opt_config = config['optimizer']
sched_config = config['lr_sched']
ema_sched_config = config['ema_sched'] # ema = esponentially moving average


opt_config['lr'] = 0.002
opt_config['weight_decay'] = 0.00005

# autoencoder checkpoint path
path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/checkpoint.pth'

# initialize model
gvad_model = K.models.denoiser_model.GVADModel(
    13,
)

inner_model = gvad_model.ae

model_denoiser = K.config.make_denoiser_wrapper(config)(inner_model)
model_ema = deepcopy(model_denoiser)


# load model state to denoiser model
checkpoint = torch.load(path)
gvad_model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']

model_ema.load_state_dict(checkpoint['model_ema'])

opt = optim.AdamW(gvad_model.parameters(),
                  lr=opt_config['lr'],
                  betas=tuple(opt_config['betas']),
                  eps=opt_config['eps'],
                  weight_decay=opt_config['weight_decay'])

# scheduler
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,
                                                  mode='min',
                                                  factor=0.1,
                                                  patience=8,
                                                  cooldown=0,
                                                  verbose=True)




ema_sched = K.utils.EMAWarmup(power=ema_sched_config['power'],
                              max_value=ema_sched_config['max_value'])


opt.load_state_dict(checkpoint['optimizer_state_dict'])
sched.load_state_dict(checkpoint['sched'])
ema_sched.load_state_dict(checkpoint['ema_sched'])


@eval_mode(model_ema)
@torch.no_grad()
def evaluate(model, sigma_min, sigma_max, loader, n_start, model_ema):

    tqdm.write('Evaluating...')

    sigmas = K.sampling.get_sigmas_karras(10, sigma_min, sigma_max, rho=7., device=device)
    sigmas = sigmas[n_start:]

    sigmas = sigmas.cuda()

    # noise is extracted randomly
    #sample_noise = torch.randn([1, 13], device=device) * sigma_max

    # noise is extracted by file
    sample_noise = seed_noise * sigmas[0]

    sample_noise = sample_noise.cuda()


    def sample_fn(x_real, sigmas, model, model_ema):

        x_real = x_real.cuda()
        x_real = torch.transpose(x_real, 2,1)

        x = sample_noise + x_real

        x = x.cuda()
        sigmas = sigmas.cuda()
        model = model.cuda()
        model_ema = model_ema.cuda()

        x_0 = K.sampling.sample_lms(model_ema, x, sigmas, disable=True)

        return x_0

    def compute_eval_outs_aot(sample_fn, dl,sigmas,model):
        outputs = []
        test_loss = []



        for batch in tqdm(dl):

            feat = batch[0]


            x_0 = sample_fn(feat,sigmas,model, model_ema)

            feat = torch.transpose(feat, 2,1)
            feat = feat.cuda()
            x_0 = x_0.cuda()

            loss =  model.loss(feat, x_0).mean().item()
            print("Loss batch", loss)

            test_loss.append(loss)
            outputs.append(x_0)

            '''
            # save trajectory to csv file
            key = str(batch[1])
            x_0 = x_0.cpu()
            x_0 = x_0.detach().numpy()
            x = x_0[0][0]
            y = x_0[0][1]
            z = x_0[0][2]


            df = pd.DataFrame(list(zip(x, y, z)), columns=['x','y','ratio_wh'])

            file_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/Plot_SameButT/'+ 'MR'  + str(n_start) + '.csv'
            df.to_csv(file_path, index=False)
            '''


        g_dists = torch.cat(outputs)
        test_loss =  sum(test_loss)/len(test_loss)
        print("loss total: ", test_loss)

        return g_dists


    gen_preds = compute_eval_outs_aot(sample_fn, loader,sigmas,model)


dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/Plot/MR/'
dataset = CustomDataset(dataset_path)

loader = DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=0, pin_memory=True)
print(len(dataset))

ssd_mean = -4
ssd_std = 0.5
sigma_min, sigma_max = find_sigma(ssd_mean, ssd_std)
for i in range(10):
  print(" Evaluating with t = ", i)
  evaluate(model=gvad_model, sigma_min=sigma_min, sigma_max=sigma_max, loader=loader, n_start=i, model_ema = model_ema)

# Learned Features


In [ ]:
''' Setting same noise. If not evaluation is different every time as the noise is randomly extracted each time '''
seed_noise_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/Noise/noise.pth'
if os.path.exists(seed_noise_path):
      seed_noise = torch.load(seed_noise_path, map_location=device)
else:
      seed_noise = torch.randn([1, 13], device=device)
      torch.save(seed_noise, seed_noise_path)

In [ ]:
'''   imports   '''
import pandas as pd
from tqdm import tqdm
import k_diffusion.models.auto_1
import k_diffusion as K
import k_diffusion.config as con


config_file_name = 'configs/config_ano.json'
config = K.config.load_config(open(config_file_name))

model_config = config['model']
dataset_config = config['dataset']
opt_config = config['optimizer']
sched_config = config['lr_sched']
ema_sched_config = config['ema_sched'] # ema = esponentially moving average


opt_config['lr'] = 0.002
opt_config['weight_decay'] = 0.00005

# autoencoder checkpoint path
path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/checkpoint.pth'

# initialize model
gvad_model = K.models.auto_1.GVADModel(
    13,
)



#model, opt, train_dl = accelerator.prepare(gvad_model, opt, loader)
inner_model = gvad_model.ae

model_denoiser = K.config.make_denoiser_wrapper(config)(inner_model)
model_ema = deepcopy(model_denoiser)



# load model state to autoencoder
checkpoint = torch.load(path)
gvad_model.load_state_dict(checkpoint['model_state_dict'])
epoch = checkpoint['epoch']

#model = con.make_denoiser_wrapper(config)(model)
model_ema.load_state_dict(checkpoint['model_ema'])

opt = optim.AdamW(gvad_model.parameters(),
                  lr=opt_config['lr'],
                  betas=tuple(opt_config['betas']),
                  eps=opt_config['eps'],
                  weight_decay=opt_config['weight_decay'])

# scheduler
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt,
                                                  mode='min',
                                                  factor=0.1,
                                                  patience=8,
                                                  cooldown=0,
                                                  verbose=True)




ema_sched = K.utils.EMAWarmup(power=ema_sched_config['power'],
                              max_value=ema_sched_config['max_value'])


opt.load_state_dict(checkpoint['optimizer_state_dict'])
sched.load_state_dict(checkpoint['sched'])
ema_sched.load_state_dict(checkpoint['ema_sched'])


@eval_mode(model_ema)
@torch.no_grad()
def evaluate(model, sigma_min, sigma_max, loader, n_start, model_ema, dataset):

    tqdm.write('Evaluating...')

    sigmas = K.sampling.get_sigmas_karras(10, sigma_min, sigma_max, rho=7., device=device)
    sigmas = sigmas[n_start:]

    sigmas = sigmas.cuda()

    # noise is extracted randomly
    #sample_noise = torch.randn([1, 13], device=device) * sigma_max

    # noise is extracted by file
    sample_noise = seed_noise * sigmas[0]

    sample_noise = sample_noise.cuda()


    def sample_fn(x_real, sigmas, model, model_ema):

        x_real = x_real.cuda()
        x_real = torch.transpose(x_real, 2,1)

        x = sample_noise + x_real

        x = x.cuda()
        sigmas = sigmas.cuda()
        model = model.cuda()
        model_ema = model_ema.cuda()

        x_0, hidden = K.sampling.sample_lms(model_ema, x, sigmas, disable=True)


        return x_0, hidden

    def compute_eval_outs_aot(sample_fn, dl,sigmas,model):
        outputs = []
        test_loss = []



        label_list = []
        hidden_repr = []

        for batch in tqdm(dl):


            feat = batch[0]
            labels = batch[1]
            #print("labels", labels)

            #print("feat", feat)


            x_0, hidden = sample_fn(feat,sigmas,model, model_ema)


            label_list = labels.tolist()

            hidden_repr = hidden[0].tolist()


            feat = torch.transpose(feat, 2,1)
            feat = feat.cuda()
            x_0 = x_0.cuda()

            #print("x_0", x_0.mean().item())



            loss =  model.loss(feat, x_0).mean().item()
            #print("Loss batch", loss)

            test_loss.append(loss)
            outputs.append(x_0)

            #print("x_0", x_0)



            # save trajectory to csv file

            key = str(batch[1])

            #print("KEY", key)


            '''
            x_0 = x_0.cpu()
            x_0 = x_0.detach().numpy()
            x = x_0[0][0]
            y = x_0[0][1]
            z = x_0[0][2]


            df = pd.DataFrame(list(zip(x, y, z)), columns=['x','y','ratio_wh'])

            file_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/Plot_SameButT/'+ 'MR'  + str(n_start) + '.csv'
            df.to_csv(file_path, index=False)
            '''




        #print("output len", len(outputs))

        g_dists = torch.cat(outputs)

        #print("g_dists", g_dists.size())

        #print("OUTPUT")
        #print(outputs[-1].size())

        #print(outputs)

        test_loss =  sum(test_loss)/len(test_loss)
        print("loss total: ", test_loss)

        return g_dists, label_list, hidden_repr


    gen_preds, labels, hiddens = compute_eval_outs_aot(sample_fn, loader,sigmas,model)


    '''
    label_names = []


    for label in labels:
      #print("label,", label)
      label_name = str(dataset.int_to_label(int(label)))
      label_names.append(label_name)


    hidden_repr = []
    for hidden in hiddens:
      #hidden = hidden.cpu()
      #hidden = hidden.detach().numpy()
      #print("hidden", type(hidden))
      a = str(hidden).replace("[", "")
      a = a.replace("]","")
      a = a.replace("  ", " ")
      a = a.strip()
      hidden_repr.append(a)



    df = pd.DataFrame(list(zip(label_names,hidden_repr)), columns=['label','features'])

    print(df)

    file_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_'+ str(n_start) + "/" + 'test' + '.csv'
    df.to_csv(file_path, index=False)

    #print("gen_preds", gen_preds)
    #print("gen_preds", gen_preds.size())
    '''

dataset_path = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/SLB/'
dataset = CustomDataset(dataset_path)
print(len(dataset))


loader= DataLoader(dataset, batch_size=len(dataset), shuffle=False, num_workers=0, pin_memory=True)


ssd_mean = -4
ssd_std = 0.5
sigma_min, sigma_max = find_sigma(ssd_mean, ssd_std)
for i in range(10):
  print(" Evaluating with t = ", i)
  evaluate(model=gvad_model, sigma_min=sigma_min, sigma_max=sigma_max, loader=loader, n_start=i, model_ema = model_ema, dataset = dataset)

# MLP


## Datasets and Utils

In [33]:
'''   Dataset   '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch

'''   custom dataset with learned features    '''
class CustomDatasetFeatures(Dataset):
    def __init__(self, dataset_path):

        self.data = []
        self.learned_feats = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(dataset_path)

    def __len__(self):
        return len(self.learned_feats)


    def __getitem__(self, idx):
        class_id, learn_feat = self.data[idx]

        learn_feat = torch.Tensor(learn_feat)

        # return both trajectory and class id
        return learn_feat, class_id

    def _init_dataset(self, dataset_path):

        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        self.labels_codec.fit(list_behaviors)

        df = pd.read_csv(dataset_path)
        self.labels = [self.label_to_int(i) for i in df['label'].tolist()]

        self.learned_feats = df['features'].tolist()

        for label, feat in zip(self.labels, self.learned_feats):

          feat = list(feat.split(" "))
          code = []

          for l in feat:
            if l != '':
                if '\n' in l:
                  l = l.replace('\n','')
                if ',' in l:
                  l = l.replace(',','')

                code.append(float(l))

          data = [label, code]
          self.data.append(data)


    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

In [21]:
'''   utils   '''

class Metric(object):
    def __init__(self, name):
        self.name = name
        self.sum = torch.tensor(0.)
        self.n = torch.tensor(0.)

    def update(self, val):
        self.sum += val.detach().cpu()
        self.n += 1

    @property
    def avg(self):
        return self.sum / self.n

def random_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def log(s, nl=True):
    print(s, end='\n' if nl else '')

In [22]:
'''   Focal Loss with fixed alpha    '''
'''https://discuss.pytorch.org/t/focal-loss-for-imbalanced-multi-class-classification-in-pytorch/61289/9'''
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, outputs, targets):
        min = 1e-8
        max = 1 - min
        outputs = torch.clamp(outputs, min, max)

        ce_loss = torch.nn.functional.cross_entropy(outputs, targets, reduction='none') # important to add reduction='none' to keep per-batch-item loss
        ce_loss = ce_loss.cpu()
        pt = torch.exp(-ce_loss)
        targets = targets.cpu()
        focal_loss = (self.alpha * (1-pt)**self.gamma * ce_loss).mean() # mean over the batch

        return focal_loss

In [23]:
'''   Focal Loss with alpha equal to inverse of class frequencies   '''
'''https://saturncloud.io/blog/how-to-use-class-weights-with-focal-loss-in-pytorch-for-imbalanced-multiclass-classification/#:~:text=Focal%20loss%20works%20by%20down,performance%20on%20the%20minority%20class.'''
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        min = 1e-8
        max = 1 - min
        inputs = torch.clamp(inputs, min, max)
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')

        pt = torch.exp(-ce_loss)
        targets = targets.cpu()
        pt = pt.cpu()
        ce_loss = ce_loss.cpu()
        loss = (self.alpha[targets] * (1 - pt) ** self.gamma * ce_loss).mean()
        return loss

In [24]:
'''   MLP model   '''

import torch.nn as nn

class MLP(nn.Module):
    def __init__(
            self,
            encoder_latent_space,
            output_classes
    ):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(encoder_latent_space, 256),
            nn.PReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.PReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 64),
            nn.PReLU(),
            nn.Dropout(0.1),
            nn.BatchNorm1d(64),
            nn.Linear(64, output_classes),
        )

    def forward(self, x):
        return self.mlp(x)

In [25]:
'''   Dataset Smote  '''

'''     imports     '''
import os
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import torch

'''   custom dataset with learned features    '''
class CustomDatasetSmote(Dataset):
    def __init__(self, x, y):

        self.data = []
        self.learn_feats = []
        self.labels = []

        self.labels_codec = LabelEncoder()

        self._init_dataset(x, y)

    def __len__(self):
        return len(self.learn_feats)


    def __getitem__(self, idx):
        class_id, learn_feat = self.data[idx]

        learn_feat = torch.Tensor(learn_feat)

        # return both trajectory and class id
        return learn_feat, class_id

    def _init_dataset(self, x, y):

        list_behaviors = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']
        self.labels_codec.fit(list_behaviors)

        for label, feat in zip(y,x):
            data = [label, feat]
            self.data.append(data)

        self.labels = y
        self.learn_feats = x


    def label_to_int(self, label):
        value = self.labels_codec.transform([label])
        return value[0]

    def int_to_label(self, value):
        label = self.labels_codec.inverse_transform([value])
        return label[0]

## Train

In [34]:
'''  imports   '''
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score


'''   training    '''
def train(model, optimizer, scheduler, criterion, epochs, device, train_loader, test_loader, save_weights_name, batch_size, save_model=False, wandb_update=False):
    model.to(device)


    for e in range(epochs):

        '''       training    '''
        print('training')
        model.train()

        trainF = Metric('trainF')

        progress_bar = tqdm(train_loader, desc='description')

        overall_preds = []
        overall_targets = []
        softmax = torch.nn.Softmax(dim=1)


        for code, targets in progress_bar:

            code = code.to(device)
            targets = targets.to(device)

            # forward
            outputs = model(code)

            # zero grad
            optimizer.zero_grad()

            # compute loss
            loss = criterion(outputs, targets)

            loss.div_(math.ceil(float(len(code)) / batch_size))

            # softmax tha outputs and take the max index (argmax), that is the final predicted class
            preds = torch.argmax(softmax(outputs), dim=1).cpu()

            # save results and targets for accuracy metrics
            overall_preds += preds.tolist()
            overall_targets += targets.cpu().tolist()

            # backpropagate the error
            loss.backward()

            # clip gradient
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # update weights
            optimizer.step()

            # update loss
            trainF.update(loss)


        model.eval()
        train_accuracy = accuracy_score(overall_targets, overall_preds)
        train_f1 = f1_score(overall_targets, overall_preds, average='macro')


        # save loss for epoch
        train_log = {'Train F': trainF.avg.item(),
                     'Train A': train_accuracy,
                     'Train F1': train_f1}

        log('--------------------')
        log('Epoch: {}'.format(e))
        log('Train Focal Loss:{}'.format(train_log['Train F']))
        log('Train Accuracy:{}'.format(train_accuracy))
        log('Train F1 score:{}'.format(train_f1))


        '''       test      '''
        print("test")
        model.eval()
        testF = Metric('testF')

        with torch.no_grad():
          progress_bar = tqdm(test_loader, desc='description')
          overall_preds = []
          overall_targets = []
          softmax = torch.nn.Softmax(dim=1)

          for code, targets in progress_bar:
              code = code.to(device)
              targets = targets.to(device)

              # forward
              outputs = model(code)

              # compute testing loss
              loss = criterion(outputs, targets)

              # softmax tha outputs and take the max index (argmax), that is the final predicted class
              preds = torch.argmax(softmax(outputs), dim=1).cpu()

              # save results and targets for accuracy metrics
              overall_preds += preds.tolist()
              overall_targets += targets.cpu().tolist()

              testF.update(loss)

          test_accuracy = accuracy_score(overall_targets, overall_preds)
          test_f1 = f1_score(overall_targets, overall_preds, average='macro')

          geo_w = geometric_mean_score(overall_targets, overall_preds, average='weighted')
          geo_m = geometric_mean_score(overall_targets, overall_preds, average='macro')

          test_log = {'Test F': testF.avg.item(),
                       'Test A': test_accuracy,
                      'Test F1': test_f1,
                      'Test Geo W': geo_w,
                      'Test Geo M': geo_m}

          log('Test Focal Loss:{}'.format(test_log['Test F']))
          log('Test Accuracy:{}'.format(test_log['Test A']))
          log('Test F1:{}'.format(test_log['Test F1']))
          log('Test Geo Weighted:{}'.format(test_log['Test Geo W']))
          log('Test Geo Macro:{}'.format(test_log['Test Geo M']))

        '''
        # confusion matrix
        if test_f1 >= 0.5:
          classes = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']


          cf_matrix = confusion_matrix(overall_targets, overall_preds)
          print("confusion matrix")
          df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                              columns = [i for i in classes])
          print("data frame")
          plt.figure(figsize = (12,7))
          sn.heatmap(df_cm, annot=True)
          #plt.show()
          #plt.savefig('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/ConfusionMatrix' + str(test_f1) +  '.png')

        '''
        '''
        # save model
        if test_f1 >= 0.5:
            path = "/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/MLP/"
            #name = "MLP" + str(100. * train_log['Train A'])
            name = "MLPSmote" + str(100. * test_f1)
            torch.save({
                'epoch': e+1,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict()
            },f'{path}/{name}.pth')
        '''

        # update wandb
        if wandb_update:
          wandb.log({'Epochs': e + 1,
                    'Train Focal Loss': train_log['Train F'],
                     'Train Accuracy': train_log['Train A'],
                     'Train F1': train_log['Train F1'],
                     'Test Focal Loss': test_log['Test F'],
                     'Test Accuracy': test_log['Test A'],
                     'Test F1': test_log['Test F1'],
                     'Test Geo Weighted': test_log['Test Geo W'],
                     'Test Geo Macro':test_log['Test Geo M']})

        scheduler.step(train_log['Train F'])

In [38]:
'''   imports  '''
from tqdm import tqdm
import imblearn
from array import array
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE

'''   main    '''
def main():

        focal = False
        smote = True
        #t = 5
        lr = 5e-5
        batch_size = 16
        n_workers = 2
        epochs = 200
        wandb_update = False
        weight_decay = 0.001
        #weight_decay = 0
        gamma = 3
        hidden_space_dim = 512



        for t in range(10):

          seed = 3

          # initialize random seeds
          random_seed(seed)

          mlp = MLP(
              encoder_latent_space = hidden_space_dim,
              output_classes = 12
          )

          df_weight = pd.read_csv('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train_class_weight.csv')
          class_weights = df_weight['weight'].tolist()
          class_weights = torch.FloatTensor(class_weights)

          if focal:
            criterion = FocalLoss(alpha=class_weights, gamma=gamma)

          else:
            criterion = nn.CrossEntropyLoss()

          optimizer = optim.Adam(mlp.parameters(), lr = lr, weight_decay=weight_decay)
          scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                                  mode='min',
                                                                  factor=0.1,
                                                                  patience=10,
                                                                  cooldown=0,
                                                                  verbose=True)

          device = "cuda:0"


          if wandb_update:
            # save model inputs and hyperparameters
            config = {
              "t": t,
              "lr": lr,
              "batch_size": batch_size,
              "epochs": epochs,
              "features_size": hidden_space_dim,
              "loss focal": focal,
              "smote": smote,
              "gamma": gamma,
              "weight_decay": weight_decay,
              "dropout": "dropout"
            }

            # start a new wandb run
            run = wandb.init(project='project_name', entity='entity_name', config=config, settings=wandb.Settings(start_method="thread"))

            # log gradients and model parameters
            wandb.watch(mlp)


          # load datasets
          # train dataset
          path_train = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_' + str(t) + '/train.csv'
          dataset_train = CustomDatasetFeatures(path_train)
          dataloader_train = DataLoader(dataset_train, shuffle = True, batch_size=batch_size, drop_last=True)
          print(len(dataset_train))

          # test dataset
          path_test = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_' + str(t) + '/test.csv'
          dataset_test = CustomDatasetFeatures(path_test)
          dataloader_test = DataLoader(dataset_test, shuffle = True, batch_size=batch_size, num_workers=n_workers, pin_memory=True)
          print(len(dataset_test))

          # smote
          if smote:

            to_sample = {1:33, 3: 114, 4: 18, 7: 9}

            oversample = SMOTE(k_neighbors=2, sampling_strategy=to_sample)

            x = []
            y = []
            for i in range(len(dataset_train)):
              array = dataset_train[i][0].numpy()
              x.append(array)
              y.append(dataset_train[i][1])

            from collections import Counter
            y_counter = Counter(y)
            print(y_counter)

            total_samples = len(y)


            res = np.array(x)
            res = res.reshape(13489, -1)
            res, y = oversample.fit_resample(res,y)

            from collections import Counter
            print(Counter(y))

            dataset = CustomDatasetSmote(res, y)
            dataloader_train = DataLoader(dataset, shuffle = True, batch_size=batch_size)
            print(len(dataset))



          # train
          train(
              model = mlp,
              optimizer = optimizer,
              scheduler = scheduler,
              criterion = criterion,
              epochs = epochs,
              device = device,
              train_loader = dataloader_train,
              test_loader = dataloader_test,
              save_weights_name = 'mlp.pth',
              batch_size = batch_size,
              wandb_update = wandb_update
          )

          if wandb_update:
            run.finish()

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd

In [ ]:
!wandb login

In [ ]:
main()

## Evaluation


In [ ]:
'''   imports   '''
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import matplotlib
import matplotlib.colors as mcolors
from imblearn.metrics import geometric_mean_score

'''   testing   '''
def test(model, criterion, device, data_loader, dataset, load_weights = False, file_name = None, wandb_update=False):

    if load_weights:
        checkpoint = torch.load(file_name)
        model.load_state_dict(checkpoint['model_state_dict'])
        print('Weights loaded.')


    model.eval()

    testCE = Metric('testCE')

    with torch.no_grad():
        progress_bar = tqdm(data_loader, desc='description')

        overall_preds = []
        overall_targets = []
        softmax = torch.nn.Softmax(dim=1)
        for code, targets in progress_bar:
            code = code.to(device)
            targets = targets.to(device)

            # forward
            outputs = model(code)

            # compute testing loss
            loss = criterion(outputs, targets)

            soft = softmax(outputs).cpu()

            # softmax tha outputs and take the max index (argmax), that is the final predicted class
            preds = torch.argmax(softmax(outputs), dim=1).cpu()

            # save results and targets for accuracy metrics
            overall_preds += preds.tolist()
            overall_targets += targets.cpu().tolist()

            testCE.update(loss)

        test_accuracy = accuracy_score(overall_targets, overall_preds)
        f1_w = f1_score(overall_targets, overall_preds, average='weighted')
        f1_m = f1_score(overall_targets, overall_preds, average='micro')
        f1_M = f1_score(overall_targets, overall_preds, average='macro')

        print("predictions:")
        print(overall_preds)
        print("labels")
        print(overall_targets)

        test_log = {'Test CE': testCE.avg.item(),
                      'Test A': test_accuracy}


        geo_weighted = geometric_mean_score(overall_targets, overall_preds, average='weighted')
        geo_micro = geometric_mean_score(overall_targets, overall_preds, average='micro')
        geo_macro = geometric_mean_score(overall_targets, overall_preds, average='macro')

        print("accuracy: ", test_accuracy)
        print("f1 score: ", f1_w)
        print("f1 score macro: ", f1_M)
        print("geo_weighted: ", geo_weighted)
        print("geo_micro: ", geo_micro)
        print("geo_macro: ", geo_macro)


        log('Test CE:{}'.format(test_log['Test CE']))
        log('Test Accuracy:{}'.format(test_log['Test A']))


        classes = ['C', 'EP', 'FD', 'FM', 'GC', 'JS', 'MA', 'MR', 'NB','NFM', 'S', 'SSP']


        # confusion matrix

        cf_matrix = confusion_matrix(overall_targets, overall_preds)
        print("confusion matrix")
        df_cm = pd.DataFrame(cf_matrix / np.sum(cf_matrix, axis=1)[:, None], index = [i for i in classes],
                            columns = [i for i in classes])
        print("data frame")
        plt.figure(figsize = (12,7))
        sn.heatmap(df_cm, annot=True)

        #plt.savefig('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/ConfusionMatrixBestF1Macro.png')

        plt.show()


        preds = [dataset.int_to_label(i) for i in overall_preds]
        labels = [dataset.int_to_label(i) for i in overall_targets]


        # accuracy for class

        acc = []
        n_test = []
        for c in classes:
          correct = 0
          tot = 0

          for i in range(len(labels)):
              if labels[i] == c:
                  if preds[i] == c:
                      correct += 1
                  tot += 1
          try:
              acc_class = correct/tot
          except:
              print("class has no sample")
              acc_class = "Nan"

          acc.append(acc_class)
          n_test.append(tot)


          #print("type", acc_class, type(acc_class))


        #print("accuracy classes", acc)
        n_tot = [174, 13, 14457, 48, 8, 1019, 67, 4, 198, 895, 31, 23]

        df = pd.DataFrame(list(zip( acc, n_tot, n_test,)), index=[i for i in classes], columns = [ 'Class accuracy', '# samples (tot)', '# samples (test)' ])

        plt.figure(figsize = (12,7))

        norm = mcolors.Normalize(-1,1)

        colors = [[norm(-1.0), "whitesmoke"],
            [norm( 1.0), "whitesmoke"]]

        cmap = mcolors.LinearSegmentedColormap.from_list("", colors)
        hm = sn.heatmap(df, annot=True, cmap=cmap, cbar=False, linecolor="black",fmt='g')

        plt.yticks(rotation='horizontal')
        plt.xticks()

        plt.savefig('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Table.png')
        plt.show()


In [ ]:
def eval():

    t = 5
    lr = 5e-5
    batch_size = 16
    n_workers = 2
    wandb_update = False
    weight_decay = 1e-4

    smaller_space_dim = 512

    seed = 3

    # initialize random seeds
    random_seed(seed)

    mlp = MLP(
        encoder_latent_space = smaller_space_dim,
        output_classes = 12
    )

    # extract class weights from csv file
    df_weight = pd.read_csv('/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Autoencoder/dataset/train_class_weight.csv')
    class_weights = df_weight['weight'].tolist()
    class_weights = torch.FloatTensor(class_weights)

    #criterion = FocalLoss(alpha=class_weights, gamma=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(mlp.parameters(), lr = lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                           mode='min',
                                                           factor=0.1,
                                                           patience=8,
                                                           cooldown=0,
                                                           verbose=True)

    device = "cuda:0"
    mlp.to(device)

    # load datasets
    # train dataset
    path_train = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_'+ str(t) + '/train.csv'
    dataset_train = CustomDatasetFeatures(path_train)
    dataloader_train = DataLoader(dataset_train, shuffle = True, batch_size=batch_size)
    print(len(dataset_train))

    # test dataset
    path_test = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Dataset/T_'+ str(t) + '/test.csv'
    dataset_test = CustomDatasetFeatures(path_test)
    dataloader_test = DataLoader(dataset_test, shuffle = True, batch_size=batch_size, num_workers=n_workers, pin_memory=True)
    print(len(dataset_test))

    #def test(model, criterion, device, data_loader, load_weights = False, file_name = None, wandb_update=False):
    # train
    test(
        model = mlp,
        criterion = criterion,
        device = device,
        data_loader = dataloader_test,
        dataset = dataset_test,
        file_name = '/content/drive/MyDrive/Behavior_CorkwingWrasse_CoastVision/Diffusion/Checkpoint/MLP/MLPF47.80931373868435.pth',
        load_weights = True,
    )

In [ ]:
eval()